In [1]:
# https://www.mdpi.com/2306-5354/9/7/271

import shutil
import os

In [2]:
BACTERIA_NAMES = ['ecoli', 'saureus', 'paureginosa', 'paeruginosa']

IMAGES_PATH = 'Petri_plates'
OUTPUT_PATH = 'seg_petri_plates'


def segment_images(full_images_list, output_path=OUTPUT_PATH):
    for bn in BACTERIA_NAMES:

        _family = [
            image_name for image_name in full_images_list if bn in image_name
        ]

        new_filepath = os.path.join(output_path, bn)

        os.makedirs(new_filepath, exist_ok=True)

        new_family_paths = [
            os.path.join(output_path, bn, image_name.split('/')[-1])
            for image_name in _family
        ]

        for i, orig_file in enumerate(_family):
            shutil.copy(orig_file, new_family_paths[i])


full_images_list = [
    os.path.join(IMAGES_PATH, img) for img in os.listdir(IMAGES_PATH)
]

segment_images(full_images_list)

---

In [3]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from keras.preprocessing.image import ImageDataGenerator

IMG_SHAPE = (3024 // 2, 4032 // 2, 3)
TEST_DIR = 'test_seg_petri_plates'

trdata = ImageDataGenerator()
traindata = trdata.flow_from_directory(
    directory=OUTPUT_PATH, target_size=(90, 90)
)

tsdata = ImageDataGenerator()
testdata = tsdata.flow_from_directory(
    directory=TEST_DIR, target_size=(200, 200)
)


vgg_model = VGG16(weights='imagenet', include_top=False, input_shape=IMG_SHAPE)

Found 1252 images belonging to 4 classes.
Found 35 images belonging to 4 classes.


2024-03-17 13:53:17.551247: I metal_plugin/src/device/metal_device.cc:1154] Metal device set to: Apple M1 Pro
2024-03-17 13:53:17.551290: I metal_plugin/src/device/metal_device.cc:296] systemMemory: 16.00 GB
2024-03-17 13:53:17.551299: I metal_plugin/src/device/metal_device.cc:313] maxCacheSize: 5.33 GB
2024-03-17 13:53:17.551655: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-03-17 13:53:17.552085: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 0 MB memory) -> physical PluggableDevice (device: 0, name: METAL, pci bus id: <undefined>)


In [4]:
features = vgg_model.predict(traindata)
feature_matrix = features.reshape(features.shape[0], -1)

feature_matrix

2024-03-17 13:53:21.681813: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:117] Plugin optimizer for device_type GPU is enabled.


40/40 [==============================] - 133s 3s/step


array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [5]:
traindata[0][0].shape

(32, 90, 90, 3)

In [6]:
feature_matrix[0]

array([0., 0., 0., ..., 0., 0., 0.], dtype=float32)

In [7]:
n_features = len(feature_matrix[0])
n_features

2048

---

In [8]:
from qdrant_client import QdrantClient

client = QdrantClient('localhost', port=6333)

In [9]:
from qdrant_client.http.models import Distance, VectorParams

client.create_collection(
    collection_name='test_collection',
    vectors_config=VectorParams(size=n_features, distance=Distance.DOT),
)

True

In [10]:
class_indices_ = traindata.class_indices
labels_ = traindata.labels

indices_class_ = {v: k for k, v in class_indices_.items()}

indices_class_

{0: 'ecoli', 1: 'paeruginosa', 2: 'paureginosa', 3: 'saureus'}

In [11]:
from tqdm import tqdm
from qdrant_client.http.models import PointStruct


points = [
    PointStruct(
        id=i,
        vector=vec.tolist(),
        payload={
            'label': indices_class_.get(labels_[i]),
            'img_uri': traindata.filepaths[i],
        },
    )
    for i, vec in tqdm(enumerate(feature_matrix[:]))
]

1252it [00:04, 263.64it/s]


In [12]:
operation_info = client.upsert(
    collection_name='test_collection', wait=True, points=points
)

print(operation_info)

operation_id=0 status=<UpdateStatus.COMPLETED: 'completed'>
